In [8]:
import os
username = os.environ['JUPYTERHUB_USER']
import copy

In [9]:
!pip install pyarrow
!pip install fastparquet

In [10]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='hdfs://iccluster040.iccluster.epfl.ch', port=8020, user='ebouille') # impersonate ebouille to read the file
import pandas as pd
files = hdfs.glob('/user/{0}/file/timetable/*.parquet'.format(username))
timetable = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        timetable = timetable.append(pd.read_parquet(f))
        
timetable.head(5)

,stop_id,trip_id,arrival_time,departure_time,stop_sequence,service_id,route_id,route_desc,route_type,direction_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,8503064,1.TA.26-18-j19-1.1.H,34860,34860,1,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Scheuren,47.322597,8.659544,NaN,
1,8503065:0:1,1.TA.26-18-j19-1.1.H,35100,35100,2,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Forch,47.325398,8.647964,NaN,8503065P
2,8503074,1.TA.26-18-j19-1.1.H,35160,35160,3,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Neue Forch,47.325806,8.637840,NaN,
3,8503068,1.TA.26-18-j19-1.1.H,35220,35220,4,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Maiacher,47.328169,8.631067,NaN,
4,8503066,1.TA.26-18-j19-1.1.H,35280,35280,5,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Zumikon,47.331688,8.623809,NaN,


In [11]:
files = hdfs.glob('/user/{0}/file/stops_walk/*.parquet'.format(username))
stops_walk = pd.DataFrame()
for file in files:
    with hdfs.open(file) as f:
        stops_walk = stops_walk.append(pd.read_parquet(f))

In [295]:
stops_walk.head(5)

,stop_id2,stop_name2,stop_lat2,stop_lon2,stop_id,stop_name,stop_lat,stop_lon,walk_time
0,8573711,"Zürich, Sädlenweg",47.367755,8.487480,8591214,"Zürich, In der Ey",47.369240,8.491631,424
1,8573711,"Zürich, Sädlenweg",47.367755,8.487480,8591163,"Zürich, Goldackerweg",47.371892,8.487930,553
2,8590610,"Fällanden, Schützenhaus",47.368625,8.632478,8590612,"Fällanden, Zil",47.372245,8.636214,589
3,8590610,"Fällanden, Schützenhaus",47.368625,8.632478,8590605,"Fällanden, Eggler",47.371588,8.634714,444
4,8590610,"Fällanden, Schützenhaus",47.368625,8.632478,8590752,"Pfaffhausen, Pfaffenstein",47.368558,8.628561,354


In [374]:
stops_ids = timetable['stop_id'].unique()

In [440]:
stops_walk = stops_walk[
    (stops_walk['stop_id'].isin(stops_ids))&
    (stops_walk['stop_id2'].isin(stops_ids))                       
                       ]

In [710]:
stops_ids = stops_ids.tolist()

AttributeError: 'list' object has no attribute 'tolist'

In [711]:
source = stops_ids[120]
destination = stops_ids[800]
t = 35800
K = 5
t_limit = 35800+7200

MAX_TIME = t_limit+1000

timetable_limit = timetable[
    (timetable['arrival_time'] >= t)&
    (timetable['departure_time'] <= t_limit)   
]

#optim_arrs = []
optim_star = {}


In [712]:

df_init = [(s, MAX_TIME) for s in stops_ids if s is not source]#[ (s, MAX_TIME) for s in stops_ids if s is not source]
df_init.append((source, t))

In [713]:
for s, time in df_init:
    optim_star[s] = time

In [714]:
optim_arrs = pd.DataFrame(
   df_init ,
    columns = ["stop_id", "previous_arrival_time"]  # add your column names here
)

optim_arrs = optim_arrs.set_index('stop_id')

In [715]:
mark = [source]
mark_last = []
route_considered = []
trips_considered = []
optim_arrs_list = [optim_arrs]
Journey_list = []
Journey_star = {}

In [ ]:
for k in range(1,K+1):
#for k in range(3,4):    
    print(len(mark))
    
    
    # for each marked stop p, find all routes r that serve p
    
    optim_arr_lastround = optim_arrs_list[k-1]
    
    optim_arr_curr = copy.deepcopy(optim_arr_lastround)
    
    Journey ={}
    
    print(mark)
    
    trips_mark = timetable_limit[timetable_limit['stop_id'].isin(mark)]
    
    
    trips_mark = trips_mark.merge(optim_arr_lastround,left_on='stop_id', right_on='stop_id')
    
    
    trips_mark = trips_mark[trips_mark['departure_time'] >= trips_mark['previous_arrival_time'] ]
    
    routes_for_mark = trips_mark[['route_id','direction_id','stop_id','stop_sequence']].drop_duplicates()
    
    routes_for_mark = routes_for_mark[~ routes_for_mark['route_id'].isin(route_considered)]
    
    trips_mark_total = timetable_limit[timetable_limit['trip_id'].isin(trips_mark['trip_id'])]
    
    trips_mark_total = trips_mark_total[~ trips_mark_total['trip_id'].isin(trips_considered)]
    
    
    # find the earliest trip w.r.t to the 
    earliest_trips = trips_mark_total.loc[trips_mark_total.groupby(['route_id','direction_id','stop_id'])['arrival_time'].idxmin()]
    
    # find the walk path
    walk_path = stops_walk[
        (stops_walk['stop_id' ].isin(mark))&
        (~ stops_walk['stop_id2' ].isin(mark_last))
    ]
    
    earliest_trips = earliest_trips.reset_index()
    
    mark_last = mark
    mark = []
    
    route_considered = []
    
    for index, row in routes_for_mark.iterrows():
        route_id = row['route_id']
        direction_id = row['direction_id']
        stop_id = row['stop_id']
        
        stop_seq = row['stop_sequence']
        
        route_considered.append(route_id)
        
        earliest_trips_given_route = earliest_trips[
            (earliest_trips['route_id'] == route_id) &
            (earliest_trips['direction_id'] == direction_id) &
            (earliest_trips['stop_sequence'] > stop_seq)
        ]
        
        for index2, row2 in earliest_trips_given_route.iterrows():
            stop_id_next = row2['stop_id']
            trip = row2['trip_id']
            trips_considered.append(trip)
            
            mark.append(stop_id_next)
            
            if row2['arrival_time'] < min(optim_arr_curr.at[stop_id_next, 'previous_arrival_time'], optim_star[destination]):
                optim_arr_curr.at[stop_id_next, 'previous_arrival_time'] = row2['arrival_time']
                optim_star[stop_id_next] =  row2['arrival_time']
            
                journey = {
                    'stop_id':stop_id_next,
                    'trip_id':row2['trip_id'],
                    'arrival_time':row2['arrival_time'],
                    'parent_stop_id': stop_id
                }
            
                Journey[stop_id_next] = journey
                if stop_id_next not in Journey_star:
                    Journey_star[stop_id_next] = journey
            
                elif row2['arrival_time'] < Journey_star[stop_id_next]['arrival_time']:
                    Journey_star[stop_id_next] = journey
            
    for index3, row3 in  walk_path.iterrows():
        from_stop_id = row3['stop_id']
        to_stop_id = row3['stop_id2']
        walk_time = row3['walk_time']
        
        arr_time = optim_arr_curr.at[from_stop_id, 'previous_arrival_time'] + walk_time
        if optim_arr_curr.at[to_stop_id, 'previous_arrival_time'] > arr_time:
            optim_arr_curr.at[to_stop_id, 'previous_arrival_time'] = arr_time
            journey = {
                'stop_id':to_stop_id,
                'trip_id':'walk',
                'arrival_time':arr_time,
                'parent_stop_id': from_stop_id
            }
            Journey[to_stop_id] = journey
            if stop_id_next not in Journey_star:
                Journey_star[stop_id_next] = journey
            
            elif arr_time < Journey_star[stop_id_next]['arrival_time']:
                Journey_star[stop_id_next] = journey
            
    mark = list(set(mark))
            
    Journey_list.append(Journey)
        
    optim_arrs_list.append(optim_arr_curr)
    
    #earliest_trips = earliest_trips[]

In [727]:

Journey_star[destination]

{'stop_id': '8573205:0:F',
 'trip_id': '118.TA.26-765-j19-1.1.R',
 'arrival_time': 41340,
 'parent_stop_id': '8588553'}

In [731]:
optim_arrs_list[5].at[destination,'previous_arrival_time']

36252

In [707]:
Journey_list[2]['8573205']

{'stop_id': '8573205',
 'trip_id': '543.TA.26-10-j19-1.6.R',
 'arrival_time': 40620,
 'parent_stop_id': '8588078'}

In [708]:
optim_arrs_list[3].at['8573205','previous_arrival_time']

40620

In [709]:
Journey_list[1]['8587348']

{'stop_id': '8587348',
 'trip_id': '602.TA.26-31-j19-1.14.R',
 'arrival_time': 38460,
 'parent_stop_id': '8591261'}

In [699]:
Journey_list[0]['8591233']

{'stop_id': '8591233',
 'trip_id': '258.TA.26-703-j19-1.2.R',
 'arrival_time': 37680,
 'parent_stop_id': '8590504'}

In [1039]:
source = stop_ids[100]
K = 5
Tau = []
Tau_0 = {}
Tau_star = {}
for s in stops_ids:
    Tau_0[s] = MAX_TIME
    Tau_star[s] = MAX_TIME
Tau_0[source] = t
Tau.append(Tau_0)

Parent = []

parent_0 = {}

Parent.append(parent_0)

mark = [source]

In [1040]:
for k in range(1,K+1):
    
    parent_k = {}
    last_route = []
    Tau_k = copy.deepcopy(Tau[k-1])
    Q = {}
    for p in mark:
        #print(p)
        routes = findServep(p, Tau[k-1][p])
        for (r,d) in routes:
            if (r,d) in Q:
                if isBefore(p,Q[(r,d)],r,d):
                    Q[(r,d)] = p
                    
            else: 
                Q[(r,d)] = p
                
    print(len(mark))
    mark = []
    for (r,d) in Q:
        stop = Q[(r,d)]
        
        trip_id,seq_after = findEarliestTrip(r,d,stop,Tau[k-1][stop])
        
        trip_after  =  timetable_limit[
            (timetable_limit.trip_id == trip_id)&
            (timetable_limit.stop_sequence.isin(seq_after))
        ]
        
        #stops_after = timetable_limit.stop_id.values.tolist()
        
        #arr_time_after = timetable_limit.arrival_time.values.tolist()
        
        for index,row in trip_after.iterrows():
            
            stop_after = row['stop_id']
            arr_time_after = row['arrival_time']
            dep_time_after = row['departure_time']
            
            
            if arr_time_after< min(Tau_star[stop_after],Tau_star[destination]):
                Tau_k[stop_after] = arr_time_after
                Tau_star[stop_after] = arr_time_after
                
                mark.append(stop_after)
                
                parent_k[stop_after] = {
                                        'stop_id': stop_after,
                                        'trip_id':trip_id,
                                        'arrival_time':arr_time_after,
                                        'departure_time': dep_time_after,
                                        'parent_stop_id':stop,
                                        'round':k
                                       }
                
    walk_path = stops_walk[
        (stops_walk['stop_id' ].isin(mark))&
        (~ stops_walk['stop_id2' ].isin(mark))
    ]
    
             
    for index3, row3 in  walk_path.iterrows():
        from_stop_id = row3['stop_id']
        to_stop_id = row3['stop_id2']
        walk_time = row3['walk_time']
        
        
        if Tau_k[to_stop_id] > Tau[k-1][from_stop_id]+walk_time:
            
            Tau_k[to_stop_id] = Tau[k-1][from_stop_id]+walk_time
            Tau_star[to_stop_id] = min(Tau_k[to_stop_id], Tau_star[to_stop_id])
            
            mark.append(to_stop_id)
            
            parent_k[to_stop_id] = {
                                        'stop_id': to_stop_id,
                                        'trip_id':'walk',
                                        'arrival_time':Tau_k[to_stop_id],
                                        'departure_time': Tau_k[to_stop_id],
                                        'parent_stop_id':to_stop_id,
                                        'round':k
                                       }
            

            
    mark = list(set(mark))
        
    Tau.append(Tau_k)
    Parent.append(parent_k)
        
            
        
        

1
25


KeyboardInterrupt: 

In [1022]:
len(Tau_star)

1548

In [940]:
destination = stops_ids[5]

In [941]:
Tau_star[destination]

38880

Get route

In [978]:
for k in range(1,K+1):
    if Tau[k][destination] < MAX_TIME:
        max_trans = k
        route = []
        i = max_trans
        d = destination
        print(k)
        while i > 0:
            parent_i = Parent[i]
            par = parent_i[d]
            route.append(par)
            d = par['parent_stop_id']
            i = i-1
        break;
    
    
            
        

3


In [979]:
source

'8591195'

In [980]:
route

[{'stop_id': '8503075',
  'trip_id': '38.TA.26-18-j19-1.3.R',
  'arrival_time': 38880,
  'departure_time': 38880,
  'parent_stop_id': '8503059',
  'round': 3},
 {'stop_id': '8503059',
  'trip_id': '151.TA.26-8-C-j19-1.5.H',
  'arrival_time': 37320,
  'departure_time': 37320,
  'parent_stop_id': '8591184',
  'round': 2},
 {'stop_id': '8591184',
  'trip_id': '503.TA.26-32-j19-1.4.R',
  'arrival_time': 36180,
  'departure_time': 36180,
  'parent_stop_id': '8591195',
  'round': 1}]

In [ ]:
timetable[timetable.trip_id == '503.TA.26-32-j19-1.4.R']

In [1014]:
def findServep(stop_id, arr_t):
    valid_trips = timetable_limit[
        (timetable_limit['stop_id'] == stop_id)&
        (timetable_limit['arrival_time'] >= arr_t)
    ]
    
    return  valid_trips[['route_id','direction_id']].drop_duplicates().values.tolist() 

In [1030]:
findServep(source, t)

[['26-32-j19-1', '1'], ['26-32-j19-1', '0']]

In [1031]:
def isBefore(stop_id1, stop_id2, route_id, direction_id):
    res = False
    trips = timetable_limit[(timetable_limit['route_id'] == route_id)&
                              (timetable_limit['direction_id'] == direction_id)&
                            ((timetable_limit['stop_id'] == stop_id1)|
                            (timetable_limit['stop_id'] == stop_id2))]
    if not trips.empty:
        T = {}
        for index,row in trips.iterrows():
            T[row['stop_id']] = row['stop_sequence']
        
            if (stop_id1 in T) and (stop_id2 in T):
                res = T[stop_id1] < T[stop_id2]

    return res


In [1016]:
trips = timetable_limit[(timetable_limit['route_id'] == '429.TA.26-705-j19-1.3.H')&
                              (timetable_limit['direction_id'] == '0')]

In [1017]:
trips

,stop_id,trip_id,arrival_time,departure_time,stop_sequence,service_id,route_id,route_desc,route_type,direction_id,stop_name,stop_lat,stop_lon,location_type,parent_station


In [1018]:
timetable_limit = timetable

In [1051]:
def findEarliestTrip(r,d,stop,t):
    
        earliest_trip = timetable_limit[
            (timetable_limit['route_id']==r)&
            (timetable_limit['direction_id']==d)&
            (timetable_limit['stop_id']==stop)&
            (timetable_limit['departure_time']>= t)
        ]
        
        #print(earliest_trip)
        if not earliest_trip.empty:
            earliest_trip = earliest_trip\
            .loc[earliest_trip.groupby(['route_id','direction_id','stop_id'])['arrival_time'].idxmin()]
        
        #earliest_trip.head(5)
        
            earliest_trip_id = earliest_trip.trip_id.values.tolist()[0]
        
            trip = timetable_limit[
            (timetable_limit['trip_id'] == earliest_trip_id)
            ]
        
            trip_id = trip['trip_id'].values.tolist()[0]
        
            seq = trip[trip['stop_id'] == stop]['stop_sequence'].max()
            
    
        
            trip.reset_index()
        
            trip_after = trip[trip['stop_sequence']>seq]
        
            seq_after = trip_after['stop_sequence'].values.tolist()
            
        else:
            trip_id = '123'
            seq_after = [0]
        return trip_id,seq_after
    

In [1052]:
valid_trips = timetable_limit[
        (timetable_limit['stop_id'] == source)&
        (timetable_limit['arrival_time'] >= t)
    ]
    

In [1053]:
 valid_trips[['route_id','direction_id']].drop_duplicates().values.tolist() 

[['26-32-j19-1', '1'], ['26-32-j19-1', '0']]

In [1054]:
findEarliestTrip('26-32-j19-1','0',source,t)

('1221.TA.26-32-j19-1.7.H', [22, 23, 24, 25, 26])

In [1055]:
r = '26-705-j19-1'
d = '1'


In [1056]:
earliest_trip = timetable_limit[
            (timetable_limit['route_id']==r)&
            (timetable_limit['direction_id']==d)&
            (timetable_limit['stop_id']==source)&
            (timetable_limit['departure_time']>= t)
        ]


earliest_trip = earliest_trip\
        .loc[earliest_trip.groupby(['route_id','direction_id','stop_id'])['arrival_time'].idxmin()]
        
earliest_trip.head(5)

,stop_id,trip_id,arrival_time,departure_time,stop_sequence,service_id,route_id,route_desc,route_type,direction_id,stop_name,stop_lat,stop_lon,location_type,parent_station


In [1057]:
earliest_trip_id = earliest_trip.trip_id

In [1058]:
earliest_trip_id =earliest_trip.trip_id.values.tolist()[0]


IndexError: list index out of range

In [1059]:
trip = timetable_limit[
            (timetable_limit['trip_id'] == earliest_trip_id)
].set_index('stop_id')


ValueError: Can only compare identically-labeled Series objects

In [1060]:
timetable_limit

,stop_id,trip_id,arrival_time,departure_time,stop_sequence,service_id,route_id,route_desc,route_type,direction_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,8503064,1.TA.26-18-j19-1.1.H,34860,34860,1,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Scheuren,47.322597,8.659544,NaN,
1,8503065:0:1,1.TA.26-18-j19-1.1.H,35100,35100,2,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Forch,47.325398,8.647964,NaN,8503065P
2,8503074,1.TA.26-18-j19-1.1.H,35160,35160,3,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Neue Forch,47.325806,8.637840,NaN,
3,8503068,1.TA.26-18-j19-1.1.H,35220,35220,4,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Maiacher,47.328169,8.631067,NaN,
4,8503066,1.TA.26-18-j19-1.1.H,35280,35280,5,TA+b0b46,26-18-j19-1,S-Bahn,400,0,Zumikon,47.331688,8.623809,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1172,8576788,99.TA.26-813-j19-1.2.R,55440,55440,4,TA+b0003,26-813-j19-1,Bus,700,1,"Uster, Burgreben",47.343887,8.717090,NaN,
1173,8581539,99.TA.26-813-j19-1.2.R,55500,55500,5,TA+b0003,26-813-j19-1,Bus,700,1,"Uster, Apothekerstrasse",47.346182,8.715069,NaN,
1174,8576095,99.TA.26-813-j19-1.2.R,55500,55500,6,TA+b0003,26-813-j19-1,Bus,700,1,"Uster, Stadtpark",47.348154,8.715697,NaN,
1175,8581548,99.TA.26-813-j19-1.2.R,55560,55560,7,TA+b0003,26-813-j19-1,Bus,700,1,"Uster, Poststrasse",47.349116,8.717000,NaN,


In [1072]:
def getBestRoutes(source,destination, begin_time,time_limit, max_trans):
    
    t = begin_time

    t_limit = time_limit

    MAX_TIME = t_limit+1000

    K = max_trans
    Tau = []
    Tau_0 = {}
    Tau_star = {}
    for s in stops_ids:
        Tau_0[s] = MAX_TIME
        Tau_star[s] = MAX_TIME
    Tau_0[source] = t
    Tau.append(Tau_0)

    Parent = []

    parent_0 = {}
    
    Parent.append(parent_0)

    mark = [source]
    
    for k in range(1,K+1):
    
        parent_k = {}
        last_route = []
        Tau_k = copy.deepcopy(Tau[k-1])
        Q = {}
        for p in mark:
        #print(p)
            routes = findServep(p, Tau[k-1][p])
            for (r,d) in routes:
                if (r,d) in Q:
                    if isBefore(p,Q[(r,d)],r,d):
                        Q[(r,d)] = p
                    
                else: 
                    Q[(r,d)] = p
                
        print(len(mark))
        mark = []
        for (r,d) in Q:
            stop = Q[(r,d)]
        
            trip_id,seq_after = findEarliestTrip(r,d,stop,Tau[k-1][stop])
        
            trip_after  =  timetable_limit[
                (timetable_limit.trip_id == trip_id)&
                (timetable_limit.stop_sequence.isin(seq_after))
            ]
        
            #stops_after = timetable_limit.stop_id.values.tolist()
        
            #arr_time_after = timetable_limit.arrival_time.values.tolist()
        
            for index,row in trip_after.iterrows():
            
                stop_after = row['stop_id']
                arr_time_after = row['arrival_time']
                dep_time_after = row['departure_time']
                stop_lat = row['stop_lat']
                stop_lon = row['stop_lon']
            
            
                if arr_time_after< min(Tau_star[stop_after],Tau_star[destination]):
                    Tau_k[stop_after] = arr_time_after
                    Tau_star[stop_after] = arr_time_after
                
                    mark.append(stop_after)
                
                    parent_k[stop_after] = {
                                        'stop_id': stop_after,
                                        'trip_id':trip_id,
                                        'stop_lat':stop_lat,
                                        'stop_lon':stop_lon,
                                        'arrival_time':arr_time_after,
                                        'departure_time': dep_time_after,
                                        'parent_stop_id':stop,
                                        'round':k
                                       }
                #if Tau[k-1][stop_after] < dep_time_after:
                    
                #    trip_id,seq_after = findEarliestTrip(r,d,stop_after,Tau[k-1][stop_after])
                
        walk_path = stops_walk[
            (stops_walk['stop_id' ].isin(mark))&
            (~ stops_walk['stop_id2' ].isin(mark))
        ]
    
             
        for index3, row3 in  walk_path.iterrows():
            from_stop_id = row3['stop_id']
            to_stop_id = row3['stop_id2']
            walk_time = row3['walk_time']
            stop_lat = row3['stop_lat']
            stop_lon = row3['stop_lon']
        
            if Tau_k[to_stop_id] > Tau[k-1][from_stop_id]+walk_time:
            
                Tau_k[to_stop_id] = Tau[k-1][from_stop_id]+walk_time
                Tau_star[to_stop_id] = min(Tau_k[to_stop_id], Tau_star[to_stop_id])
            
                mark.append(to_stop_id)
            
                parent_k[to_stop_id] = {
                                        'stop_id': to_stop_id,
                                        'trip_id':'walk',
                                        'stop_lat':stop_lat,
                                        'stop_lon':stop_lon,
                                        'arrival_time':Tau_k[to_stop_id],
                                        'departure_time': Tau_k[to_stop_id],
                                        'parent_stop_id':to_stop_id,
                                        'round':k
                                       }
            

            
        mark = list(set(mark))
        
        Tau.append(Tau_k)
        Parent.append(parent_k)
        
    
        
    # recover the best route
    best_route = []
    if Tau_star[destination] == MAX_TIME:
        print('no routes found')
    
    for k in range(1,K+1):
        if Tau[k][destination] < MAX_TIME:
            max_trans = k
            
            i = max_trans
            d = destination
            print(k)
            while i > 0:
                parent_i = Parent[i]
                par = parent_i[d]
                print(par)
                best_route.append(par)
                d = par['parent_stop_id']
                i = i-1
            break;
            
    return best_route
    

In [1074]:
best_route = getBestRoutes(stops_ids[100],stops_ids[1020], 35700,35700+10000,5)

1
25
250
473
222
4


KeyError: '8503020:0:2'

In [1063]:
best_route

[{'stop_id': '8591063',
  'trip_id': '1577.TA.26-14-A-j19-1.24.R',
  'stop_lat': 47.413336370854,
  'stop_lon': 8.54584397892258,
  'arrival_time': 37800,
  'departure_time': 37800,
  'parent_stop_id': '8502572',
  'round': 2},
 {'stop_id': '8502572',
  'trip_id': '503.TA.26-32-j19-1.4.R',
  'stop_lat': 47.3702920484894,
  'stop_lon': 8.51391785372053,
  'arrival_time': 35880,
  'departure_time': 35880,
  'parent_stop_id': '8591195',
  'round': 1}]

In [984]:
[1,2,3].reverse()

In [962]:
a = reversed([1,2,3])

In [963]:
a